In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.applications import DenseNet121, EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from random import randint

# Step 1: Data Loading and Preprocessing
classes_path = "C:\\Users\\Dimple\\OneDrive\\Desktop\\potato slices preprocessing\\dataset"
class_names = ['\\Processed Defected Slices', '\\Processed Potato slicess']
num_classes = len(class_names)
img_size = (224, 224, 3)  # DenseNet and EfficientNet both support this input size

In [2]:
images = []
labels = []
for cl in class_names:
    for img in os.listdir(classes_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        img_path = os.path.join(classes_path + cl, img)
        img = Image.open(img_path).resize((img_size[0], img_size[1]))
        img_array = np.asarray(img)
        images.append(img_array)

labels = np.asarray(labels)
images = np.asarray(images)

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.1, random_state=42)

In [3]:
# Step 2: Define DenseNet121 Model
def build_densenet():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=img_size)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# Step 3: Define EfficientNetB3 Model
def build_efficientnet():
    base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=img_size)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# Instantiate models
densenet_model = build_densenet()
efficientnet_model = build_efficientnet()

# Step 4: Train Models
densenet_model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_val, y_val))
efficientnet_model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_val, y_val))



43941136/43941136 [==============================] - 29s 1us/step
Epoch 1/10


18/18 [==============================] - 70s 3s/step - loss: 0.2725 - accuracy: 0.8796 - val_loss: 0.1607 - val_accuracy: 0.9344
Epoch 2/10
18/18 [==============================] - 49s 3s/step - loss: 0.0662 - accuracy: 0.9726 - val_loss: 0.0534 - val_accuracy: 0.9836
Epoch 3/10
18/18 [==============================] - 48s 3s/step - loss: 0.0332 - accuracy: 0.9909 - val_loss: 0.0271 - val_accuracy: 1.0000
Epoch 4/10
18/18 [==============================] - 50s 3s/step - loss: 0.0190 - accuracy: 0.9982 - val_loss: 0.0184 - val_accuracy: 1.0000
Epoch 5/10
18/18 [==============================] - 48s 3s/step - loss: 0.0131 - accuracy: 1.0000 - val_loss: 0.0178 - val_accuracy: 1.0000
Epoch 6/10
18/18 [==============================] - 46s 3s/step - loss: 0.0221 - accuracy: 0.9945 - val_loss: 0.0177 - val_accuracy: 1.0000
Epoch 7/10
18/18 [==============================] - 44s 2s/step - loss: 0.0081 - accuracy:

In [26]:
# Step 5: Extract Predictions for Meta-Model Training
# DenseNet predictions
densenet_preds_train = np.argmax(densenet_model.predict(X_train), axis=1).reshape(-1, 1)
densenet_preds_val = np.argmax(densenet_model.predict(X_val), axis=1).reshape(-1, 1)

# EfficientNet predictions
efficientnet_preds_train = np.argmax(efficientnet_model.predict(X_train), axis=1).reshape(-1, 1)
efficientnet_preds_val = np.argmax(efficientnet_model.predict(X_val), axis=1).reshape(-1, 1)

2/2 [==============================] - 3s 2s/step


In [27]:
# Stack the predictions to form features for XGBoost
stacked_preds_train = np.hstack([densenet_preds_train, efficientnet_preds_train])
stacked_preds_val = np.hstack([densenet_preds_val, efficientnet_preds_val])

In [30]:
# Confirm shapes for debugging
print(f"stacked_preds_train shape: {stacked_preds_train.shape}")  # Should be (num_samples, 2)
print(f"stacked_preds_val shape: {stacked_preds_val.shape}") 
# Flatten labels for meta-model (convert from one-hot to single-label format)
y_train_flat = np.argmax(y_train, axis=1)
y_val_flat = np.argmax(y_val, axis=1)

# Initialize and train the XGBoost meta-classifier
meta_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    max_depth=3,           # Lower max depth to prevent overfitting
    n_estimators=50,        # Reduce the number of trees
    learning_rate=0.1       # Moderate learning rate
)
meta_model.fit(stacked_preds_train, y_train_flat)

stacked_preds_train shape: (548, 2)
stacked_preds_val shape: (61, 2)


c:\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:52:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=50,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [31]:
# Step 7: Evaluate the Ensemble Model
final_preds = meta_model.predict(stacked_preds_val)
accuracy = accuracy_score(y_val_flat, final_preds)
print(f"Stacking Ensemble Accuracy with XGBoost: {accuracy * 100:.2f}%")

Stacking Ensemble Accuracy with XGBoost: 100.00%


In [32]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.applications import DenseNet121, EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [33]:
# Step 1: Data Loading and Preprocessing
classes_path = "C:\\Users\\Dimple\\OneDrive\\Desktop\\potato slices preprocessing\\dataset"
class_names = ['\\Processed Defected Slices', '\\Processed Potato slicess']
num_classes = len(class_names)
img_size = (224, 224, 3)

images = []
labels = []
for cl in class_names:
    for img in os.listdir(classes_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        img_path = os.path.join(classes_path + cl, img)
        img = Image.open(img_path).resize((img_size[0], img_size[1]))
        img_array = np.asarray(img)
        images.append(img_array)

labels = np.asarray(labels)
images = np.asarray(images)

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.1, random_state=42)

In [34]:
# Step 2: Define DenseNet121 Model with Dropout and Fine-Tuning
def build_densenet():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=img_size)
    for layer in base_model.layers[-20:]:  # Unfreeze last 20 layers
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [35]:
# Step 3: Define EfficientNetB3 Model with Dropout and Fine-Tuning
def build_efficientnet():
    base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=img_size)
    for layer in base_model.layers[-20:]:  # Unfreeze last 20 layers
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [36]:
# Instantiate models
densenet_model = build_densenet()
efficientnet_model = build_efficientnet()

# Step 4: Train Models
densenet_model.fit(X_train, y_train, epochs=5, verbose=1, validation_data=(X_val, y_val))
efficientnet_model.fit(X_train, y_train, epochs=5, verbose=1, validation_data=(X_val, y_val))

Epoch 1/5
18/18 [==============================] - 289s 13s/step - loss: 0.8070 - accuracy: 0.5438 - val_loss: 0.6092 - val_accuracy: 0.6885
Epoch 2/5
18/18 [==============================] - 237s 13s/step - loss: 0.5420 - accuracy: 0.7172 - val_loss: 0.5097 - val_accuracy: 0.7213
Epoch 3/5
18/18 [==============================] - 245s 14s/step - loss: 0.4475 - accuracy: 0.7920 - val_loss: 0.4189 - val_accuracy: 0.8033
Epoch 4/5
18/18 [==============================] - 179s 10s/step - loss: 0.3590 - accuracy: 0.8558 - val_loss: 0.3306 - val_accuracy: 0.9180
Epoch 5/5
18/18 [==============================] - 215s 12s/step - loss: 0.2541 - accuracy: 0.9161 - val_loss: 0.2597 - val_accuracy: 0.9344
Epoch 1/5
18/18 [==============================] - 1052s 55s/step - loss: 0.6739 - accuracy: 0.6004 - val_loss: 0.6870 - val_accuracy: 0.5082
Epoch 2/5
18/18 [==============================] - 1061s 58s/step - loss: 0.6348 - accuracy: 0.6460 - val_loss: 0.6655 - val_accuracy: 0.5082
Epoch 3/5
1

In [37]:
# Step 5: Extract Predictions for Meta-Model Training
# DenseNet predictions
densenet_preds_train = np.argmax(densenet_model.predict(X_train), axis=1).reshape(-1, 1)
densenet_preds_val = np.argmax(densenet_model.predict(X_val), axis=1).reshape(-1, 1)

# EfficientNet predictions
efficientnet_preds_train = np.argmax(efficientnet_model.predict(X_train), axis=1).reshape(-1, 1)
efficientnet_preds_val = np.argmax(efficientnet_model.predict(X_val), axis=1).reshape(-1, 1)

2/2 [==============================] - 5s 2s/step


In [38]:
# Stack the predictions to form features for XGBoost
stacked_preds_train = np.hstack([densenet_preds_train, efficientnet_preds_train])
stacked_preds_val = np.hstack([densenet_preds_val, efficientnet_preds_val])

# Confirm shapes for debugging
print(f"stacked_preds_train shape: {stacked_preds_train.shape}")  # Should be (num_samples, 2)
print(f"stacked_preds_val shape: {stacked_preds_val.shape}")      # Should be (num_samples, 2)

stacked_preds_train shape: (548, 2)
stacked_preds_val shape: (61, 2)


In [39]:
# Flatten labels for meta-model (convert from one-hot to single-label format)
y_train_flat = np.argmax(y_train, axis=1)
y_val_flat = np.argmax(y_val, axis=1)

In [40]:
# Initialize and train the XGBoost meta-classifier with adjusted parameters
meta_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    max_depth=3,            # Lower max depth to prevent overfitting
    n_estimators=50,         # Reduced number of trees
    learning_rate=0.1        # Moderate learning rate
)
meta_model.fit(stacked_preds_train, y_train_flat)


c:\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:53:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=50,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [41]:
# Step 7: Evaluate the Ensemble Model
final_preds = meta_model.predict(stacked_preds_val)
accuracy = accuracy_score(y_val_flat, final_preds)
print(f"Stacking Ensemble Accuracy with XGBoost: {accuracy * 100:.2f}%")

Stacking Ensemble Accuracy with XGBoost: 93.44%


In [42]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_val_flat, final_preds)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[27  3]
 [ 1 30]]


In [43]:
# Save the trained DenseNet model
densenet_model.save("densenet_model.h5")

# Save the trained EfficientNet model
efficientnet_model.save("efficientnet_model.h5")

# Save the XGBoost meta-model
import joblib
joblib.dump(meta_model, "xgboost_meta_model.joblib")


c:\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['xgboost_meta_model.joblib']